<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=91d84e1d19e6b06b03e7c3b5580ec4bb10a5138a797088e3dfb42227df0e0f01
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-26 10:08:52
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: 1.18 L (0.83%)
Current PnL: -22.43 L (-14.7%)
CY Booked + Current PnL: -8.54 L (-5.59%)
-------------------
Total profit:  1.51 L
Total loss:  -23.94 L
-------------------
Total Booked + Current PnL: 18.44 L (14.68%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.69%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.84 L (61.27%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.57,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,73.0,M-SC,7.93,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.07,-14.04,16.45,0.09,14250.0,-14153.0,86624.0,101.07,50.0,M-SC,3.70,253.0,-0.99,0.60,13.80,OX40N,NTT,DURABLES
35,ICICIGI,2252.93,1.07,7.79,11.51,20.19,21627.0,13577.0,187897.0,-15.55,54.0,X-MC,7.49,68.0,0.63,1.31,24.08,X40,ATH,INSURANCE
83,VOLTAS,1530.00,1.45,7.64,11.20,19.69,23115.0,14643.0,206385.0,-2.87,50.0,X-MC,3.38,78.0,0.63,1.44,14.95,XY25,NTT,AC
53,NESTLEIND,1377.00,1.08,12.57,7.84,21.40,23426.0,33369.0,298795.0,4.42,58.0,X-LC,7.79,12.0,1.42,2.08,20.05,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,2.86,-12.09,42.77,25.51,69972.0,-22495.0,163600.0,0.51,63.0,H-LC,1.86,49.0,-0.32,1.14,20.56,AR,ATH,ELECTRICAL
28,GREENPANEL,537.00,2.69,-30.71,106.58,43.14,135203.0,-56222.0,126856.0,184.24,37.0,M-SC,5.44,240.0,-0.42,0.88,19.33,XY24,NTT,MISC
39,INDIGOPNTS,1408.00,2.57,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,73.0,M-SC,7.93,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
72,TANLA,1943.92,2.52,-35.36,239.85,119.69,419814.0,-95729.0,175032.0,-34.70,34.0,H-SC,11.17,156.0,-0.23,1.22,38.27,AR,ATH,IT
0,5PAISA,593.00,2.52,-34.90,73.42,12.90,104441.0,-76245.0,142251.0,109.13,58.0,H-SC,14.00,161.0,-0.73,0.99,18.28,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.95,-27.81,97.86,42.84,45926.0,-18076.0,46930.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.12,XY24,NTT,MISC
44,JCHAC,2282.00,-1.79,-24.85,33.06,-0.01,31751.0,-31765.0,96040.0,19552.17,31.0,M-SC,1.60,233.0,-1.00,0.67,8.75,OX40N,NTT,AC
38,INDIAMART,4810.62,-1.65,-0.28,107.31,106.72,131976.0,-350.0,122986.0,-50.63,37.0,H-SC,3.37,139.0,-0.00,0.86,23.03,AR,ATH,MISC
12,BERGEPAINT,680.00,-1.40,-0.02,21.17,21.14,48116.0,-53.0,227286.0,-7.04,51.0,X-MC,3.98,74.0,-0.00,1.59,28.11,XY24,NTT,PAINTS
62,ROUTE,2227.21,-0.99,-49.79,236.67,69.05,158136.0,-66249.0,66817.0,-59.29,36.0,H-SC,23.69,140.0,-0.42,0.47,3.05,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.65,-0.28,107.31,106.72,131976.0,-350.0,122986.0,-50.63,37.0,H-SC,3.37,139.0,-0.0,0.86,23.03,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.42,-4.17,116.96,107.91,165480.0,-6160.0,141484.0,-22.17,38.0,M-SC,10.47,216.0,-0.04,0.99,1.17,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.57,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,73.0,M-SC,7.93,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.07,-14.04,16.45,0.09,14250.0,-14153.0,86624.0,101.07,50.0,M-SC,3.70,253.0,-0.99,0.60,13.80,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.27,-22.69,46.77,13.47,97508.0,-61182.0,208485.0,-68.31,27.0,H-SC,1.28,158.0,-0.63,1.45,6.01,XY24,NTT,PAINTS
67,SIS,528.00,1.03,-23.09,57.66,21.26,49242.0,-25632.0,85400.0,2016.06,51.0,H-SC,3.86,166.0,-0.52,0.60,15.38,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.09,1.85,69.14,72.27,119318.0,3127.0,172575.0,-12.43,58.0,M-LC,4.43,99.0,0.03,1.2,8.13,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.09,1.85,69.14,72.27,119318.0,3127.0,172575.0,-12.43,58.0,M-LC,4.43,99.0,0.03,1.20,8.13,XR,NTT,IT
38,INDIAMART,4810.62,-1.65,-0.28,107.31,106.72,131976.0,-350.0,122986.0,-50.63,37.0,H-SC,3.37,139.0,-0.00,0.86,23.03,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.93,-1.18,37.25,35.63,77144.0,-2475.0,207099.0,-14.97,41.0,H-MC,4.17,119.0,-0.03,1.44,15.29,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,0.41,-3.82,89.14,81.92,98427.0,-4380.0,110418.0,-43.65,37.0,M-SC,3.81,236.0,-0.04,0.77,30.72,XR,NTT,DURABLES
25,FINCABLES,1641.55,0.42,-4.17,116.96,107.91,165480.0,-6160.0,141484.0,-22.17,38.0,M-SC,10.47,216.0,-0.04,0.99,1.17,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.52,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.07,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.99,-9.05,26.69,15.23,48633.0,-18131.0,182216.0,-22.11,29.0,X-MC,6.78,66.0,-0.37,1.27,20.28,X40N,NTT,REALTY
76,TMPV,600.00,2.14,-14.69,66.67,42.18,157208.0,-40610.0,235800.0,-23.08,30.0,X-LC,1.05,3.0,-0.26,1.64,2.14,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.40,-13.81,72.23,48.45,124830.0,-27683.0,172823.0,-20.09,32.0,X-MC,7.07,64.0,-0.22,1.21,0.40,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.34,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.42,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.59,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,0.89,5.0,-0.13,1.38,3.98,X40,NTT,FMCG
76,TMPV,600.0,2.14,-14.69,66.67,42.18,157208.0,-40610.0,235800.0,-23.08,30.0,X-LC,1.05,3.0,-0.26,1.64,2.14,XY24,NTT,AUTO
3,ACC,3906.0,0.36,-21.16,108.22,64.15,203010.0,-50361.0,187590.0,-53.69,55.0,X-SC,1.59,82.0,-0.25,1.31,5.23,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,1.45,2.98,27.18,30.97,49467.0,5259.0,181997.0,-21.26,56.0,X-MC,1.69,75.0,0.11,1.27,15.91,X40,ATH,INSURANCE
66,SIEMENS,4671.5,2.86,-12.09,42.77,25.51,69972.0,-22495.0,163600.0,0.51,63.0,H-LC,1.86,49.0,-0.32,1.14,20.56,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.52,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.07,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.40,-13.81,72.23,48.45,124830.0,-27683.0,172823.0,-20.09,32.0,X-MC,7.07,64.0,-0.22,1.21,0.40,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.34,-4.54,40.12,33.76,76910.0,-9120.0,191700.0,-32.64,33.0,X-MC,3.55,57.0,-0.12,1.34,0.42,X40,ATH,FMCG
54,PAGEIND,51605.08,0.84,-5.07,32.80,26.07,50984.0,-8300.0,155440.0,-30.06,37.0,X-MC,9.19,55.0,-0.16,1.08,0.84,X40,ATH,APPARELS
56,QUESS,424.00,-1.95,-27.81,97.86,42.84,45926.0,-18076.0,46930.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.12,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.95,-27.81,97.86,42.84,45926.0,-18076.0,46930.0,-52.62,41.0,X-SC,37.47,83.0,-0.39,0.33,2.12,XY24,NTT,MISC
60,RELAXO,1176.00,1.77,-47.26,188.77,52.31,144528.0,-68597.0,76563.0,-43.86,34.0,X-SC,4.14,91.0,-0.47,0.53,2.48,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.52,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.07,X40,NTT,FOOTWEAR
22,DIXON,18931.72,1.75,-5.90,29.20,21.58,38508.0,-8262.0,131877.0,-52.56,35.0,X-MC,6.74,56.0,-0.21,0.92,16.75,X40N,ATH,IT
52,MEDANTA,1486.00,0.84,5.22,17.93,24.08,23949.0,6621.0,133571.0,-3.24,50.0,X-SC,5.19,89.0,0.28,0.93,25.85,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,0.62,-14.72,39.87,19.28,117628.0,-50928.0,295028.0,-25.57,60.0,X-LC,7.37,1.0,-0.43,2.06,9.07,X40,ATH,IT
41,INFY,2275.00,1.05,5.29,47.10,54.87,155160.0,16538.0,329426.0,-18.16,59.0,X-LC,2.87,2.0,0.11,2.30,14.01,X40,BTT,IT
76,TMPV,600.00,2.14,-14.69,66.67,42.18,157208.0,-40610.0,235800.0,-23.08,30.0,X-LC,1.05,3.0,-0.26,1.64,2.14,XY24,NTT,AUTO
82,VBL,671.64,1.30,-8.11,47.65,35.67,138293.0,-25616.0,290226.0,-19.05,47.0,X-LC,6.02,4.0,-0.19,2.02,4.62,X40N,ATH,FMCG
43,ITC,452.00,0.59,-1.50,12.12,10.44,23893.0,-2998.0,197140.0,-41.15,44.0,X-LC,0.89,5.0,-0.13,1.38,3.98,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.80,-36.87,110.51,32.89,53860.0,-28468.0,48738.0,-701.92,64.0,L-MC,5.60,259.0,-0.53,0.34,34.22,XR,NTT,BANKS
6,ASIANTILES,137.00,0.94,-11.90,106.20,81.67,87775.0,-11159.0,82651.0,7433.33,55.0,L-SC,19.07,271.0,-0.13,0.58,60.06,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.53,-12.82,60.06,39.54,52968.0,-12968.0,88191.0,-31.83,64.0,M-SC,6.62,220.0,-0.24,0.62,24.36,AR,ATH,AUTO
51,MASFIN,398.61,0.72,-5.13,28.65,22.05,26632.0,-5025.0,92955.0,-18.25,50.0,H-SC,7.41,164.0,-0.19,0.65,35.12,XR,ATH,FINANCE
70,SURYODAY,216.00,0.91,-18.83,51.57,23.03,74962.0,-33711.0,145360.0,57.89,50.0,H-SC,10.15,167.0,-0.45,1.01,44.23,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.57,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,73.0,M-SC,7.93,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.58,1.10,19.62,20.95,49943.0,2781.0,254549.0,0.07,77.0,X-LC,13.98,31.0,0.06,1.78,37.13,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,1.80,-36.87,110.51,32.89,53860.0,-28468.0,48738.0,-701.92,64.0,L-MC,5.60,259.0,-0.53,0.34,34.22,XR,NTT,BANKS
0,5PAISA,593.00,2.52,-34.90,73.42,12.90,104441.0,-76245.0,142251.0,109.13,58.0,H-SC,14.00,161.0,-0.73,0.99,18.28,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,0.67,5.78,14.17,20.77,35096.0,13536.0,247680.0,-3.38,61.0,X-MC,3.69,62.0,0.39,1.73,13.10,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,44.56
2,50,75.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.70
MC    29.20
LC    25.11
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.89
X40      21.18
X40N     11.88
XR        9.65
XY25      9.07
AR        9.03
OX40N     7.70
X200      1.84
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.00
X-MC    22.78
X-LC    19.96
M-SC    12.04
X-SC     8.24
H-MC     4.71
H-LC     2.98
L-SC     1.42
M-MC     1.37
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,-4.76,38.64
IT,13.72,-14.98,74.34
FINANCE,9.46,-14.38,62.71
MISC,7.44,-21.91,71.85
PAINTS,5.95,-8.29,24.37
ELECTRICAL,5.95,-10.02,50.32
INSURANCE,4.46,1.03,33.85
PHARMA,3.87,-0.59,32.78
AUTO,3.34,-20.08,71.57


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3037301.0,21
XR,1279129.0,13
AR,1277772.0,10
X40,1004541.0,14
X40N,745397.0,9
OX40N,687381.0,10
XY25,339857.0,6
SR,279699.0,2
MH,72542.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3541262.0,25
M-SC,1330672.0,15
X-MC,1218012.0,16
X-LC,901928.0,11
X-SC,738945.0,8
H-MC,401985.0,3
L-SC,258647.0,3
H-LC,130317.0,2
M-LC,119318.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1209903.0      6
           AR         873715.0      5
           XR         786838.0      7
M-SC       XY24       778422.0      6
X-MC       X40        436285.0      7
X-LC       X40        393712.0      5
X-MC       XY24       357319.0      3
H-SC       OX40N      318565.0      4
X-SC       X40N       291516.0      3
M-SC       OX40N      281580.0      5
H-SC       SR         279699.0      2
X-SC       XY24       272885.0      3
X-MC       X40N       247067.0      4
X-LC       XY24       227904.0      2
H-MC       AR         211117.0      2
X-LC       X40N       206814.0      2
H-MC       XY24       190868.0      1
X-MC       XY25       177341.0      2
X-SC       X40        174544.0      2
L-SC       XR         171411.0      2
M-SC       XR         147702.0      2
           AR         122968.0      2
M-LC       XR         119318.0      1
L-SC       OX40N       87236.0      1
X-LC       XY25        73498.0      2
H-SC       MH          72542.0      1
H-LC       AR          69972.0      1
           X200        60345.0      1
L-MC       XR          53860.0      1
M-MC       XY25        48431.0      1
L-LC       XY25        40587.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
